In [33]:
from bs4 import BeautifulSoup
import requests
import time

In [34]:
url = 'https://www.boxofficemojo.com/chart/ww_top_lifetime_gross/?area=XWW'
movies = []
imdb_ids = []

In [35]:
# main function, returns all relevant information we are looking for

def soupy(url, movies, imdb_ids):
    time.sleep(5)
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    movies.extend(movie_deets(soup, movies))
    imdb_ids.extend(imdb_tags(soup, imdb_ids))
    print(len(imdb_ids))
    back = next_url(soup)
    if back:
        front = url.split('/chart')[0]
        new_url = front + back
        print(new_url)
        return soupy(new_url,movies, imdb_ids)
    else:
        return imdb_ids, movies
        
        

In [36]:
def imdb_tags(soup, imdb_ids):
    movie_info = soup.findAll('tr', style='')
    imdb_id = []
    for i in range(1,201):
        for detail in movie_info[i].find('td', class_='a-text-left mojo-field-type-title'):
            imdb_id.append(detail.get('href').split('/')[2]) 
    return imdb_id
    

In [37]:
# gets movie details in list format and returns to main function

def movie_deets(soup, movie_list):
    # navigates to where list of movies is
    movie_info = soup.findAll('tr', style='')
    movies = []
    
    # loops through all movies in the list and adds info to master movie list
    for i in range(1,201):
        movie_deets = []
        for detail in movie_info[i].findAll('td'):
            movie_deets.append(detail.text)
        movies.append(movie_deets)
    return movies
    

In [38]:
# gets the next url to navigate to the next page of results

def next_url(soup):
    
    #navigates to where url is
    next_ur = soup.find('li', class_='a-last').find('a') 
    
    #if url is there will return the next url
    # once the end of results is reached will return False to main function in order to terminate it
    if next_ur:
        return next_ur.get('href')
    else:
        return False
        

In [39]:
soupy(url, movies, imdb_ids)

200
https://www.boxofficemojo.com/chart/ww_top_lifetime_gross/?offset=200&area=XWW
400
https://www.boxofficemojo.com/chart/ww_top_lifetime_gross/?offset=400&area=XWW
600
https://www.boxofficemojo.com/chart/ww_top_lifetime_gross/?offset=600&area=XWW
800
https://www.boxofficemojo.com/chart/ww_top_lifetime_gross/?area=XWW&offset=800
1000


(['tt4154796',
  'tt0499549',
  'tt0120338',
  'tt2488496',
  'tt4154756',
  'tt0369610',
  'tt6105098',
  'tt0848228',
  'tt2820852',
  'tt4520988',
  'tt2395427',
  'tt1825683',
  'tt1201607',
  'tt2527336',
  'tt4881806',
  'tt2294629',
  'tt2771200',
  'tt3606756',
  'tt4630562',
  'tt1300854',
  'tt2293640',
  'tt3498820',
  'tt1477834',
  'tt0167260',
  'tt6320628',
  'tt4154664',
  'tt1399103',
  'tt1074638',
  'tt2109248',
  'tt1345836',
  'tt7286456',
  'tt2527338',
  'tt1979376',
  'tt0435761',
  'tt0383574',
  'tt3748528',
  'tt6139732',
  'tt1298650',
  'tt3469046',
  'tt0107290',
  'tt2277860',
  'tt0120915',
  'tt1014759',
  'tt2948356',
  'tt0903624',
  'tt0468569',
  'tt0241527',
  'tt0926084',
  'tt1690953',
  'tt0110357',
  'tt3040964',
  'tt2283362',
  'tt0449088',
  'tt1170358',
  'tt2310332',
  'tt0167261',
  'tt0373889',
  'tt0266543',
  'tt0417741',
  'tt0298148',
  'tt1727824',
  'tt0330373',
  'tt0413300',
  'tt0120737',
  'tt1080016',
  'tt2379713',
  'tt22509

In [40]:
df = pd.DataFrame(movies)

In [41]:
df.head()

,0,1,2,3,4,5,6,7
0,1,Avengers: Endgame,"$2,797,800,564","$858,373,000",30.7%,"$1,939,427,564",69.3%,2019
1,2,Avatar,"$2,790,439,000","$760,507,625",27.2%,"$2,029,931,375",72.8%,2009
2,3,Titanic,"$2,194,439,542","$659,363,944",30%,"$1,535,075,598",70%,1997
3,4,Star Wars: Episode VII - The Force Awakens,"$2,068,223,624","$936,662,225",45.3%,"$1,131,561,399",54.7%,2015
4,5,Avengers: Infinity War,"$2,048,359,754","$678,815,482",33.1%,"$1,369,544,272",66.9%,2018


In [42]:
def get_columns(url):
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    
    headers = soup.find('div', class_='a-section imdb-scroll-table-inner')
    heads = headers.find('table').findAll('th')
    
    columns = []
    for head in heads:
        columns.append(head.text.split('\n')[0])
    return columns
    

In [43]:
df.columns = get_columns(url)

In [44]:
df.head()

,Rank,Title,Worldwide Lifetime Gross,Domestic Lifetime Gross,Domestic %,Foreign Lifetime Gross,Foreign %,Year
0,1,Avengers: Endgame,"$2,797,800,564","$858,373,000",30.7%,"$1,939,427,564",69.3%,2019
1,2,Avatar,"$2,790,439,000","$760,507,625",27.2%,"$2,029,931,375",72.8%,2009
2,3,Titanic,"$2,194,439,542","$659,363,944",30%,"$1,535,075,598",70%,1997
3,4,Star Wars: Episode VII - The Force Awakens,"$2,068,223,624","$936,662,225",45.3%,"$1,131,561,399",54.7%,2015
4,5,Avengers: Infinity War,"$2,048,359,754","$678,815,482",33.1%,"$1,369,544,272",66.9%,2018


In [45]:
df['imdb_id'] = imdb_ids

In [49]:
df.head()

,Rank,Title,Worldwide Lifetime Gross,Domestic Lifetime Gross,Domestic %,Foreign Lifetime Gross,Foreign %,Year,imdb_id
0,1,Avengers: Endgame,"$2,797,800,564","$858,373,000",30.7%,"$1,939,427,564",69.3%,2019,tt4154796
1,2,Avatar,"$2,790,439,000","$760,507,625",27.2%,"$2,029,931,375",72.8%,2009,tt0499549
2,3,Titanic,"$2,194,439,542","$659,363,944",30%,"$1,535,075,598",70%,1997,tt0120338
3,4,Star Wars: Episode VII - The Force Awakens,"$2,068,223,624","$936,662,225",45.3%,"$1,131,561,399",54.7%,2015,tt2488496
4,5,Avengers: Infinity War,"$2,048,359,754","$678,815,482",33.1%,"$1,369,544,272",66.9%,2018,tt4154756


In [50]:
df.shape

(1000, 9)

In [51]:
import csv

In [52]:
df.to_csv('web_scraping_info.csv')